In [249]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV # CV -crossvalueation약자(교차검증약자)

train = pd.read_csv("./data/train.csv",index_col="no")
test = pd.read_csv("./data/test.csv",index_col="no")
submission = pd.read_csv("./data/sample_submission.csv",index_col="no")



def strip_title(row):
    return row.strip()

# train 공백제거
# except pass 는 수치형나오면 오류나는거를 pass한것
for i in range(len(train.columns)):
    try:
        train.iloc[:,i] = train.iloc[:,i].apply(strip_title)
    except:
        pass
# test 공백제거
for i in range(len(test.columns)):
    try:
        test.iloc[:,i] = test.iloc[:,i].apply(strip_title)
    except:
        pass
    

In [164]:
# ?을 other로 변환
train['occupation'] = train['occupation'].replace('?','other')
train['workclass'] = train['workclass'].replace('?','other')
train['native-country'] = train['native-country'].replace('?','United-States')

# ?을 other로 변환
test['occupation'] = test['occupation'].replace('?','other')
test['workclass'] = test['workclass'].replace('?','other')
test['native-country'] = test['native-country'].replace('?','United-States')

# 맵핑

## 맵핑 dic

### workclass

In [165]:
# workclass
w_dic = {
'Private': 'Private', 
'Self-emp-not-inc': 'Self-emp-not-inc', 
'Local-gov': 'gov', 
'other': 'other',
'State-gov':'gov' ,
'Self-emp-inc':'Self-emp-inc' ,
'Federal-gov':'gov' ,
'Without-pay': 'Private',
'Never-worked': 'Private'
}
train['workclass'] = train['workclass'].map(w_dic)
test['workclass'] = test['workclass'].map(w_dic)

### education

In [ ]:
ed_dic={
    'HS-grad' : 1,
    'Some-college': 2,
    'Bachelors': 3,
    'Masters': 4,
    'Assoc-voc': 5,
    '11th': 6,
    'Assoc-acdm': 7,
    '10th': 8,
    '7th-8th':9,
    'Prof-school': 10,
    '9th':11,
    '12th':12,
    '5th-6th':13,
    'Doctorate':14,
    '1st-4th':15,
    'Preschool':16
}
X_train['education'] = X_train['education'].map(ed_dic)
X_test['education'] = X_test['education'].map(ed_dic)

### education-num

In [166]:
# education-num
en_dic={
    1 : 1,
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 1,
    9: 2,
    10: 2,
    11:3,
    12:3,
    13:3,
    14:4,
    15:4,
    16:4
}
train['education-num_label'] = train['education-num'].map(en_dic)
test['education-num_label'] = test['education-num'].map(en_dic)

### marital-status

In [167]:

ms_dic = {
'Divorced':'Divorced',
'Never-married':'Never-married',
'Married-civ-spouse':'Married-civ-spouse',
'Separated': 'Other',
'Married-spouse-absent':'Other',
'Widowed':'Other',
'Married-AF-spouse':'Other'
}
train['marital-status']=train['marital-status'].map(ms_dic)
test['marital-status']=test['marital-status'].map(ms_dic)

### occupation

In [168]:
oc_dic={
'Machine-op-inspct': 'a',
'Other-service':'b',
'Handlers-cleaners':'b',
'Tech-support':'c',
'Transport-moving':'d',
'Farming-fishing':'e',
'Prof-specialty':'f',
'Priv-house-serv':'b',
'Adm-clerical':'a',
'Protective-serv':'c',
 'Exec-managerial':'f',
 '?':'e',
 'Craft-repair':'d',
 'Sales':'c',
 'Armed-Forces':'c'
}
train['occupation']=train['occupation'].map(oc_dic)
test['occupation']=test['occupation'].map(oc_dic)

### relationship

In [169]:

re_dic={
'Not-in-family':'Not-in-family',
'Husband':'Husband',
'Unmarried':'m',
'Other-relative':'m',
'Own-child':'m',
'Wife':'Wife'
}
train['relationship']=train['relationship'].map(re_dic)
test['relationship']=test['relationship'].map(re_dic)

### race

In [170]:
# race

ra_dic = {
    'White': 'White',
    'Black' : 'Black',
    'Asian-Pac-Islander':'other',
    'Amer-Indian-Eskimo': 'other',
    'Other' : 'other'
}
train['race']=train['race'].map(ra_dic)
test['race']=test['race'].map(ra_dic)

### native-country

In [171]:
# native-country

train['native-country'] = train['native-country'].map({'United-States':'United-States'})
train['native-country'].fillna("other-country",inplace=True)
test['native-country'] = test['native-country'].map({'United-States':'United-States'})
test['native-country'].fillna("other-country",inplace=True)

# 스케일링

In [172]:
from sklearn.preprocessing import StandardScaler
# 스케일러 생성 (평균과 표준편차)
std_scaler = StandardScaler()
# 기준값 생성 (평균,표준편차)

## fnlwgt

In [173]:
# 항상 2차원으로 넣어야함(DataFrame 형태)
std_scaler.fit(train[["fnlwgt"]])
# 데이터변형
scaled_fnlwgt_train = std_scaler.transform(train[["fnlwgt"]])
scaled_fnlwgt_test = std_scaler.transform(test[["fnlwgt"]])
train['scaled_fnlwgt']=scaled_fnlwgt_train
test['scaled_fnlwgt']=scaled_fnlwgt_test

# CUT 수치->범주

## age

In [174]:
# bins = [0,20,40,60,80,100]
# # 데이터를 잘라낼 구간정보 설정 -1~0 1구간,(0+1)~3 2구간 (3+1)~20 3구간
# labels = ["1","2","3",'4','5'] # 구간별 범주 이름
# cut_result_train = pd.cut(x=train['age'], # 구간화할 데이터 설정
#                    bins=bins, # 잘라낼 구간정보 설정
#                    labels=labels) # 구간별 범주 이름 설정

# cut_result_test = pd.cut(x=test['age'], # 구간화할 데이터 설정
#                    bins=bins, # 잘라낼 구간정보 설정
#                    labels=labels) # 구간별 범주 이름 설정


# train['age_b']=cut_result_train.astype('int')
# test['age_b']=cut_result_test.astype('int')


In [175]:
# train['age_b']

# 인코딩전 드랍

In [176]:
## fnlwgt -> 스케일정규화 후 컬럼추가 scaled_fnlwgt
train.drop('fnlwgt',axis=1,inplace=True)
test.drop('fnlwgt',axis=1,inplace=True)

# train.drop('scaled_fnlwgt',axis=1,inplace=True)
# test.drop('scaled_fnlwgt',axis=1,inplace=True)

# train.drop('capital-gain',axis=1,inplace=True)
# test.drop('capital-gain',axis=1,inplace=True)

# train.drop('capital-loss',axis=1,inplace=True)
# test.drop('capital-loss',axis=1,inplace=True)

## education 과 상관관계가 높아서 드랍
train.drop('education',axis=1,inplace=True)
test.drop('education',axis=1,inplace=True)

## education-num -> 구간별 합친후 education-num_label
# train.drop('education-num',axis=1,inplace=True)
# test.drop('education-num',axis=1,inplace=True)

## age -> cut 함수 이용후 수치->범주 age_b
# train.drop('age',axis=1,inplace=True)
# test.drop('age',axis=1,inplace=True)

In [177]:
train.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,education-num_label,scaled_fnlwgt
no,,,,,,,,,,,,,,,
1,25,Private,7,Divorced,a,Not-in-family,White,Male,0,0,40,United-States,0,1,0.278922
2,39,Private,10,Divorced,b,Not-in-family,White,Female,0,1721,55,United-States,0,2,-1.294649
3,35,Private,13,Never-married,b,Not-in-family,other,Female,0,0,50,other-country,0,3,0.067813
4,64,Private,11,Divorced,c,Not-in-family,White,Female,0,0,40,United-States,0,3,-0.513179
5,24,Private,10,Married-civ-spouse,d,Husband,White,Male,0,0,70,United-States,1,2,-1.220761


In [178]:
test.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,education-num_label,scaled_fnlwgt
no,,,,,,,,,,,,,,
29306,18,other,10,Never-married,NaN,m,White,Male,0,0,16,United-States,2,0.525767
29307,29,Private,9,Married-civ-spouse,b,Wife,White,Female,0,0,40,United-States,2,-1.010410
29308,45,Private,13,Married-civ-spouse,c,Husband,Black,Male,0,0,40,United-States,3,-1.463555
29309,42,Self-emp-not-inc,13,Married-civ-spouse,f,Husband,White,Male,0,0,40,United-States,3,-1.184333
29310,41,Private,16,Married-civ-spouse,f,Wife,White,Female,0,1902,40,United-States,4,0.057608


In [179]:
train.columns

Index(['age', 'workclass', 'education-num', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
       'hours-per-week', 'native-country', 'income', 'education-num_label',
       'scaled_fnlwgt'],
      dtype='object')

In [180]:
test.columns

Index(['age', 'workclass', 'education-num', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
       'hours-per-week', 'native-country', 'education-num_label',
       'scaled_fnlwgt'],
      dtype='object')

In [181]:
set(train.columns)-set(test.columns)

{'income'}

In [182]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29305 entries, 1 to 29305
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  29305 non-null  int64  
 1   workclass            29305 non-null  object 
 2   education-num        29305 non-null  int64  
 3   marital-status       29305 non-null  object 
 4   occupation           27637 non-null  object 
 5   relationship         29305 non-null  object 
 6   race                 29305 non-null  object 
 7   sex                  29305 non-null  object 
 8   capital-gain         29305 non-null  int64  
 9   capital-loss         29305 non-null  int64  
 10  hours-per-week       29305 non-null  int64  
 11  native-country       29305 non-null  object 
 12  income               29305 non-null  int64  
 13  education-num_label  29305 non-null  int64  
 14  scaled_fnlwgt        29305 non-null  float64
dtypes: float64(1), int64(7), object(7)
m

In [183]:
train.select_dtypes(object).columns

Index(['workclass', 'marital-status', 'occupation', 'relationship', 'race',
       'sex', 'native-country'],
      dtype='object')

# 인코딩

In [184]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29305 entries, 1 to 29305
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  29305 non-null  int64  
 1   workclass            29305 non-null  object 
 2   education-num        29305 non-null  int64  
 3   marital-status       29305 non-null  object 
 4   occupation           27637 non-null  object 
 5   relationship         29305 non-null  object 
 6   race                 29305 non-null  object 
 7   sex                  29305 non-null  object 
 8   capital-gain         29305 non-null  int64  
 9   capital-loss         29305 non-null  int64  
 10  hours-per-week       29305 non-null  int64  
 11  native-country       29305 non-null  object 
 12  income               29305 non-null  int64  
 13  education-num_label  29305 non-null  int64  
 14  scaled_fnlwgt        29305 non-null  float64
dtypes: float64(1), int64(7), object(7)
m

In [185]:
# object 컬럼만 가져오기
train.select_dtypes(object).columns

Index(['workclass', 'marital-status', 'occupation', 'relationship', 'race',
       'sex', 'native-country'],
      dtype='object')

In [186]:
# categorical_features = ["workclass","marital-status","occupation","relationship","race",'sex','native-country']
categorical_features = train.select_dtypes(object).columns

In [187]:
# 반복문으로 인코딩하기
for feature_name in categorical_features :
    one_hot = pd.get_dummies(train[feature_name], prefix=feature_name) 
    # prefix : 컬럼명 앞에 라벨(똑같은 클래스가 있으면 중복되기때문)
    train = pd.concat([train,one_hot],axis=1) # 기존 데이터 끝에 one_hot을 붙임
    train.drop(feature_name,axis=1,inplace=True) # 기존 글자컬럼을 삭제

In [188]:
# test 도 똑같이적용
for feature_name in categorical_features :
    one_hot = pd.get_dummies(test[feature_name], prefix=feature_name) 
    # prefix : 컬럼명 앞에 라벨(똑같은 클래스가 있으면 중복되기때문)
    test = pd.concat([test,one_hot],axis=1) # 기존 데이터 끝에 one_hot을 붙임
    test.drop(feature_name,axis=1,inplace=True) # 기존 글자컬럼을 삭제

In [189]:
train.columns

Index(['age', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'income', 'education-num_label', 'scaled_fnlwgt',
       'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_gov', 'workclass_other',
       'marital-status_Divorced', 'marital-status_Married-civ-spouse',
       'marital-status_Never-married', 'marital-status_Other', 'occupation_a',
       'occupation_b', 'occupation_c', 'occupation_d', 'occupation_e',
       'occupation_f', 'relationship_Husband', 'relationship_Not-in-family',
       'relationship_Wife', 'relationship_m', 'race_Black', 'race_White',
       'race_other', 'sex_Female', 'sex_Male', 'native-country_United-States',
       'native-country_other-country'],
      dtype='object')

In [190]:
train.shape, test.shape

((29305, 34), (19537, 33))

# 인코딩 후 드랍

In [191]:
train.drop('workclass_other',axis=1,inplace=True)
test.drop('workclass_other',axis=1,inplace=True)

# 특성 라벨 분리

In [192]:
X_train = train.drop("income",axis=1)
y_train = train['income']

In [193]:
X_train.shape, y_train.shape

((29305, 32), (29305,))

# 앙상블 랜덤포레스트

In [194]:
# 0.8609793550588638

# rf_model = RandomForestClassifier(n_estimators=200, # 배깅할 모델 갯수
#                                   max_features=0.7, # 각 모델이 랜덤하게 사용할 특성수(비율),만약 너무 높다면 다양한 의견이 줄어들수있음
#                                   max_depth=15, # 각 모델의 최대 깊이(기본 DecisionTree에서 사용되는 파라미터를 다 사용가능)
#                                   min_samples_leaf=20,
#                                   random_state=721 #난수고정
#                                  )
# # 앙상블 모델은 파라미터를 전부 고려해서 튜닝을 해줘야함 (예를들어 4중 for문 필요 그래서 그리드서치 사용)

In [195]:
# 0.8616277085821531

# rf_model = RandomForestClassifier(n_estimators=200, # 배깅할 모델 갯수
#                                   max_features=0.5, # 각 모델이 랜덤하게 사용할 특성수(비율),만약 너무 높다면 다양한 의견이 줄어들수있음
#                                   max_depth=15, # 각 모델의 최대 깊이(기본 DecisionTree에서 사용되는 파라미터를 다 사용가능)
#                                   min_samples_leaf=20,
#                                   random_state=721 #난수고정
#                                  )
# # 앙상블 모델은 파라미터를 전부 고려해서 튜닝을 해줘야함 (예를들어 4중 for문 필요 그래서 그리드서치 사용)

In [196]:
# # 0.86650 캐글 점수
# rf_model = RandomForestClassifier(n_estimators=200, # 배깅할 모델 갯수
#                                   max_features=0.5, # 각 모델이 랜덤하게 사용할 특성수(비율),만약 너무 높다면 다양한 의견이 줄어들수있음
#                                   max_depth=15, # 각 모델의 최대 깊이(기본 DecisionTree에서 사용되는 파라미터를 다 사용가능)
#                                   min_samples_leaf=20,
#                                   random_state=721 #난수고정
#                                  )
# # 앙상블 모델은 파라미터를 전부 고려해서 튜닝을 해줘야함 (예를들어 4중 for문 필요 그래서 그리드서치 사용)

In [197]:
# rf_rs = cross_val_score(rf_model,X_train,y_train,cv=5)

In [198]:
# rf_rs

In [199]:
# rf_model.fit(X_train,y_train)

In [200]:
# pre = rf_model.predict(test)

In [201]:
# submission["income"] = pre
# submission.to_csv('data/no1_pre.csv')

# XGB부스트

## 0721 전처리 전 실행

In [202]:
# 0.8692714553830404

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=200, # 앙상블 모델 갯수
#                           learning_rate=0.3, # 학습률
#                           max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [203]:
# 0.8710117727350282

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=200, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [204]:
# 0.8643576181538986

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=500, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 10, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [205]:
# 0.8701245521242109

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=500, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 5, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [206]:
# 0.8708752772564408

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=500, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 5, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.4 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [207]:
# 0.8716260023886709

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=500, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.5 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [208]:
# 0.8716260023886709

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.5 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [209]:
# 0.870090428254564

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.3, # 학습률
#                       a    max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.5 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [210]:
# 0.871796621736905

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.5 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [211]:
# 0.8724449752601945

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.7 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [212]:
# 0.871796621736905

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [213]:
# 0.8712847636922026

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.7 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [214]:
# 0.8712847636922026

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [215]:
# 0.8724449752601945

# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.7, # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                           random_state = 721
#                          )

In [216]:
# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=1000, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2 # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                          )

In [217]:
# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                          )

In [218]:
# param_grid = {
#     'max_depth' : [3,4,5],
#     'learning_rate' : [0.1,0.2,0.3],
#     'n_estimators' : [500,850,1000],
#     'gamma' : [0.2,0.4,0.8]
# }

# # 튜닝할 모델, 사용할 파라미터, 교차검증 횟수
# grid = GridSearchCV(xgb_model,param_grid,cv=5)
# # n_jobs : cpu 코어수가 많으면 n_jobs로 지정해서 여러 코어 사용가능 -1을 하면 모든 리소르를 끌어옴

In [219]:
# # 튜닝할 모델, 사용할 파라미터, 교차검증 횟수
# grid = GridSearchCV(xgb_model,param_grid,cv=5)
# # n_jobs : cpu 코어수가 많으면 n_jobs로 지정해서 여러 코어 사용가능 -1을 하면 모든 리소르를 끌어옴

In [220]:
# grid.fit(X_train,y_train)

In [221]:
# grid.best_estimator_

In [222]:
# grid.best_index_

In [223]:
# grid.best_params_

In [224]:
# grid.best_score_

In [225]:
# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=500, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2, # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                           random_state = 721
#                          )

In [226]:
# xgb_model.fit(X_train,y_train)

In [227]:
# grid.best_estimator_

In [228]:
# best_model = grid.best_estimator_
# pre = best_model.predict(X_test)

In [229]:
# xgb_cv = cross_val_score(xgb_model,X_train,y_train,cv=5)
# xgb_cv.mean()

In [230]:
# xgb_cv

In [231]:
# xgb_model.fit(X_train,y_train)

In [232]:
# pre = xgb_model.predict(test)

In [233]:
# submission["income"] = pre
# submission.to_csv('data/xgb_model_r02_pre.csv')

-
-xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
-                          n_estimators=500, # 앙상블 모델 갯수
-                          learning_rate=0.1, # 학습률
-                          max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
-                          gamma = 0.2, # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
-                          random_state = 721
-                         )
-

    - 0.87755 best(캐글), 0.872444975260194(스코어)

## 0722 실행

In [234]:
from sklearn.model_selection import GridSearchCV

In [235]:
param_grid= {
    'max_depth' : [10,12,14],
    'n_estimators' : [100,120,150],
    'max_features' : [0.6,0.7,0.75],
    'min_samples_leaf' : [6,7,9]
}

In [236]:
model = RandomForestClassifier(random_state=917)
grid = GridSearchCV(model, param_grid, cv=6, n_jobs = -1) 

In [237]:
grid.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=500, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2, # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
#                           random_state = 721
#                          )

In [ ]:
# xgb_model.fit(X_train,y_train)

In [238]:
xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
                          n_estimators=600, # 앙상블 모델 갯수
                          learning_rate=0.15, # 학습률
                          max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
                          gamma = 0.8, # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
                          random_state =791,
                          min_child_weight=0.9
                         )
pre= cross_val_score(xgb_model,X_train,y_train)
pre.mean()

KeyboardInterrupt: 

In [239]:
xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
                          n_estimators=400, # 앙상블 모델 갯수
                          learning_rate=0.2, # 학습률
                          max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
                          gamma = 0.6, # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
                          random_state =791,
                          min_child_weight=2
                         )
pre= cross_val_score(xgb_model,X_train,y_train)
pre.mean()

0.8712847636922028

In [ ]:
xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
                          n_estimators=1000, # 앙상블 모델 갯수
                          learning_rate=0.22, # 학습률
                          max_depth = 4, # 최대깊이(일반적으로 깊게 설정x)
                          gamma = 0.7, # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡)
                          random_state =791,
                          min_child_weight=1.8
                         )
pre= cross_val_score(xgb_model,X_train,y_train)
pre.mean()

In [241]:
xgb_model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.6, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=2,
              missing=nan, monotone_constraints='()', n_estimators=400,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=791,
              reg_alpha=0, reg_lambda=1, ...)

In [242]:
pre = xgb_model.predict(test)

In [248]:
y_train

no
1        0
2        0
3        0
4        0
5        1
        ..
29301    0
29302    0
29303    0
29304    0
29305    0
Name: income, Length: 29305, dtype: int64

In [ ]:
# xgb_model.fit(X_train,y_train)

In [ ]:
# xgb_model = XGBClassifier(booster='gbtree',
#                           random_state = 721)

In [ ]:
param_grid = {
    'max_depth' : [3,4,5,6],
    'learning_rate' : [0.1],
    'n_estimators' : [500,200],
    'gamma' : [0.2,0.4,0.1]
}


In [ ]:
# 튜닝할 모델, 사용할 파라미터, 교차검증 횟수
# grid = GridSearchCV(xgb_model,param_grid,cv=5)
# n_jobs : cpu 코어수가 많으면 n_jobs로 지정해서 여러 코어 사용가능 -1을 하면 모든 리소르를 끌어옴

In [ ]:
# grid.fit(X_train,y_train)

In [ ]:
# grid.best_params_

- grid.best_params {'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}

In [ ]:
# grid.best_score_

In [ ]:
# # 탐색하고 싶은 하이퍼파라미터 조합을 딕셔너리로 생성
# param_grid = {
#     'max_depth' : [i for i in range(1,21,3)],
#     'n_estimators' : [i for i in range(100,201,100)],
#     'max_features' : [0.5,0.7],
#     'min_samples_leaf' : [i for i in range(100,201,100)]
# }

In [ ]:
# model = RandomForestClassifier(random_state=721)
# # 튜닝할 모델, 사용할 파라미터, 교차검증 횟수
# grid = GridSearchCV(model,param_grid,cv=3, n_jobs=-1)
# # n_jobs : cpu 코어수가 많으면 n_jobs로 지정해서 여러 코어 사용가능 -1을 하면 모든 리소르를 끌어옴

In [ ]:
# grid.fit(X_train,y_train)